# Дескрипторы

Дескриптор атрибута определяет поведение при доступе, изменении и удалении атрибута.  
Позволяет контролировать значение конкретного атрибута.  
В отличии от декораторов (getter, setter, deleter) и декоратора-геттера @property которые реализованы этой же концепцией дескрипторов, сам дескриптор находится и реализуется в отдельном классе.

## Пример дескриптора № 1

In [2]:
class Descriptor():

    def __get__(self, obj, obj_type):
        print('get attribute') # Здесь может быть любая логика со значением атрибута

    def __set__(self, obj, value):
        print('set attribute') # Здесь может быть любая логика со значением атрибута

    def __delete__(self, obj):
        print('delete attribute') # Здесь может быть любая логика со значением атрибута


class Class():
    myattr = Descriptor() # Использование дескриптора


obj = Class()

Обращаемся к атрибуту:

In [6]:
obj.myattr

get attribute


Устанавливаем значение атрибуту:

In [7]:
obj.myattr = 1

set attribute


Удаляем атрибут:

In [8]:
del obj.myattr

delete attribute


## Пример дескриптора № 2

In [9]:
class Value():

    def __init__(self):
        self.value = None

    @staticmethod
    def _prepare_value(value):
        return value * 10

    def __get__(self, obj, obj_type):
        return self.value

    def __set__(self, obj, value):
        self.value = self._prepare_value(value)

    def __delete__(self):
        pass


class Class():
    myattr = Value()


obj = Class()

Установим значение атрибуту:

In [10]:
obj.myattr = 10

Проверим значение обратившись к атрибуту:

In [11]:
obj.myattr

100

Значение равно `100`, хотя мы устанавливали равным `10`, это произошло потому, что в реализации дескриптора была применена некотороая логика установки значения атрибуту.

## Пример дескриптора № 3

Пример реализует запись в файл какого-то важного значения, которое изменяется:

In [12]:
import random

class ImportantValue():

    def __init__(self, amount):
        self.amount = amount

    def __get__(self, obj, obj_type):
        return self.amount

    def __set__(self, obj, value):
        with open('log.txt', 'a') as f:
            f.write('Пример дескриптора #3. Новое значение amount = {}\n'.format(value))
        self.amount = value


class Account():
    # Какой-то важный атрибут, изменение которого должно логироваться
    amount = ImportantValue(100)


a = Account()
a.amount = random.randint(100, 1000)

Проверим лог-файл на содержимое:

In [13]:
with open('log.txt', 'r') as f:
    print(f.read())

Пример дескриптора #3. Новое значение amount = 784



Попробуем еще раз изменить значение атрибута и прочитаем заново лог-файл:

In [14]:
a.amount = random.randint(100, 1000)

with open('log.txt', 'r') as f:
    print(f.read())

Пример дескриптора #3. Новое значение amount = 784
Пример дескриптора #3. Новое значение amount = 604



## Функции и методы

На самом деле, функции и методы реализованы с помощью дескрипторов:

In [15]:
class Class():
    def method(self):
        pass


obj = Class()

В bound метод передается объект, с которым вызван метод:

In [16]:
obj.method

<bound method Class.method of <__main__.Class object at 0x7f13353c4d68>>

In [17]:
Class.method

<function __main__.Class.method>

## Декоратор `@property`

In [18]:
class User():
    
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    @property # Используем метод как get-атрибут
    def full_name(self):
        return '{} {}'.format(self.first_name, self.last_name)


amy = User('Amy', 'Jones')

Метод `full_name()` используется как обычный атрибут благодаря декоратору-дескриптору `@property`:

In [19]:
amy.full_name

'Amy Jones'

In [20]:
User.full_name

## Собственный класс который эмулирует поведение `@property`

In [26]:
class Property():

    def __init__(self, getter):
        self.getter = getter  # В этом атрибуте находится метод класса ClassA()

    def __get__(self, obj, obj_type=None):
        if obj is None:
            return self
        return self.getter(obj) # Вызов метода класса ClassA() с передачей в него self-объекта

Таким образом, мы можем использовать как встроенный декоратор `property()`, так и только что созданный `Property()`:

In [27]:
class ClassA():

    # 1. Используется встроенный декоратор-дескриптор
    @property
    def original_property(self):
        return 'original property'

    # 2. Используется собственный декоратор-дескриптор.
    # В метод __init__ класса Property() в качестве второго аргумента передается этот метод
    @Property
    def custom_property(self):
        return 'custom property'

    def custom_pure(self):
        return 'custom pure'

    # 3. Атрибут custom_pure. В Property передается метод custom_pure(). Используется как вызов функции.
    custom_pure = Property(custom_pure)


obj = ClassA()

Осуществляем вызовы:

In [28]:
obj.original_property

'original property'

In [29]:
obj.custom_property

'custom property'

In [30]:
obj.custom_pure

'custom pure'

## `@staticmethod` и `@classmethod` реализованы через дескрипторы

In [31]:
class StaticMethod():

    def __init__(self, func):
        self.func = func

    def __get__(self, obj, obj_type=None):
        print('Вызван метод __get__ класса StaticMethod')
        return self.func


class ClassMethod():

    def __init__(self, func):
        self.func = func

    def __get__(self, obj, obj_type=None):
        print('Вызван метод __get__ класса ClassMethod')
        if obj_type is None:
            obj_type = type(obj)

        def new_func(*args, **kwargs):
            return self.func(obj_type, *args, **kwargs)

        return new_func

Примеры использования:

In [32]:
class ClassA():

    @StaticMethod
    def methodA(a):
        print('Это methodA класса StaticMethod. attr =', a)

    @ClassMethod
    def methodB(cls, a):
        print('Это ClassMethod. attr =', a)


obj = ClassA()

Вызовы:

In [33]:
obj.methodA('call from object')    # staticmethod

Вызван метод __get__ класса StaticMethod
Это methodA класса StaticMethod. attr = call from object


In [34]:
ClassA.methodA('call from class')  # staticmethod

Вызван метод __get__ класса StaticMethod
Это methodA класса StaticMethod. attr = call from class


In [35]:
obj.methodB('call from object')    # classmethod

Вызван метод __get__ класса ClassMethod
Это ClassMethod. attr = call from object


In [36]:
ClassA.methodB('call from class')  # classmethod

Вызван метод __get__ класса ClassMethod
Это ClassMethod. attr = call from class


## `__slots__` также работает с помощью дескрипторов

* Позволяет опеределить в классе жестко заданный набор атрибутов.  
* Реализуется с помощью определения дескрипторов для каждого из атрибутов.

In [38]:
class ClassA():
    pass

class ClassB():
    __slots__ = ['anakin'] # В этом классе может быть только атрибут anakin

    def __init__(self):
        self.anakin = 'the chosen one' # Доступ к атрибуту через __slots__


Поведение для обычного класса:

In [39]:
a = ClassA()
a.luke = 'the chosen two'  # Установить новый атрибут в объекте удается
print(a.__dict__)          # Словарь существует

{'luke': 'the chosen two'}


Поведение для класса со слотом:

In [40]:
b = ClassB()
b.luke = 'the chosen two' # Установить новый атрибут в объекте НЕ удается

AttributeError: 'ClassB' object has no attribute 'luke'

Словарь не доступен:

In [41]:
b.__dict__

AttributeError: 'ClassB' object has no attribute '__dict__'

Зато существует магический атрибут `__slots__`:

In [42]:
b.__slots__

['anakin']